In [25]:
# coding: utf-8
import warnings
warnings.filterwarnings('ignore')
import os
ope = os.path.exists
opj = os.path.join
import sys
import time
import cv2
import socket
import argparse
import time
import shutil
import copy
import math
import pandas as pd
import numpy as np
import itertools as it
# import albumentations as A
# import albumentations.torch as AT
from collections import OrderedDict
from multiprocessing import cpu_count
# from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from torch.nn import BCEWithLogitsLoss, DataParallel
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data.sampler import Sampler, RandomSampler, SequentialSampler
from torch.optim.optimizer import Optimizer
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.autograd import Variable

In [26]:
RESULT_DIR = '../output/result'
DATA_DIR = '../input'
PRETRAINED_DIR = '../input/pretrained'
PI  = np.pi
INF = np.inf
EPS = 1e-12
ID = 'ImageId'
SPLIT = 'class_count'
TARGET = 'EncodedPixels'
IMG_SIZE = (256, 1600)
CROP_ID = 'CropImageId'
MASK_AREA = 'MaskArea'
DATASET = 'dataset'

STEEL = 'Steel'

In [27]:
loss_names = ['SymmetricLovaszLoss', 'BCEWithLogitsLoss']

config = {
    'out_dir': opj('.'), # destination where trained network should be saved
    'gpu_id': '0', # gpu id used for training 
    'arch': 'unet_resnet34_cbam_v0a', # model architecture
    'loss': 'SymmetricLovaszLoss', # loss function
    'scheduler': 'Adam3', # scheduler name
    'epochs': 10, # number of total epochs to run
    'img_size': (256, 1600), # image size
    'batch_size': 2, # train mini-batch size
    'workers': cpu_count(), # number of data loading workers
    'split_type': 'split', # split type options
    'split_name': 'random_folds10', # split name options
    'fold': 0, # index of fold
    'clipnorm': 1, # clip grad norm
    'resume': None, # name of the latest checkpoint
    'crop_version': None, # the cropped version
    'is_balance': 1, # is_balance
    'sample_times': 3,
    'ema': False,
    'ema_decay': 0.9999,
    'ema_start': 0,
    'pseudo': None, # pseudo type, such as chexpert_pseudo
    'pseudo_ratio': 1, # pseudo ratio selected for each epoch
    'train_transform': 'augment_default' # train augmentation list
}

In [28]:
def train_multi_augment9(image, mask):
    augment_func_list = [
        augment_default,
        augment_fliplr,
        augment_random_rotate,
        augment_random_crop,
        augment_random_cover,
        augment_random_brightness_multiply,
        augment_random_brightness_shift,
        augment_random_Gaussian,
    ]
    c = np.random.choice(len(augment_func_list))
    image, mask = augment_func_list[c](image, mask)

    return image, mask

def train_multi_augment1(image, mask):
    augment_func_list = [
        augment_default,
        augment_fliplr,
        augment_flipud,
    ]
    c = np.random.choice(len(augment_func_list))
    image, mask = augment_func_list[c](image, mask)

    return image, mask

def train_multi_augment2(image, mask):
    augment_func_list = [
        augment_default,
        augment_fliplr,
        augment_flipud,
        augment_random_crop,
    ]
    c = np.random.choice(len(augment_func_list))
    image, mask = augment_func_list[c](image, mask)

    return image, mask

###########################################################################################

def augment_default(image, mask=None):
    return image, mask

def unaugment_default(prob):
    return prob

def augment_fliplr(image, mask=None):
    image = np.fliplr(image)
    if mask is not None:
        mask = np.fliplr(mask)
    return image, mask

def unaugment_fliplr(prob):
    prob = np.fliplr(prob)
    return prob

def augment_flipud(image, mask=None):
    image = np.flipud(image)
    if mask is not None:
        mask = np.flipud(mask)
    return image, mask

def unaugment_flipud(prob):
    prob = np.flipud(prob)
    return prob

def augment_random_brightness_shift(image, mask=None, limit=0.2):
    alpha = np.random.uniform(-1 * limit, limit) * image.max()

    image = image + alpha
    image = np.clip(image, 0, 255).astype('uint8')
    return image, mask

def augment_random_brightness_multiply(image, mask=None, limit=0.2):
    alpha = np.random.uniform(-1 * limit, limit)
    image = image * (1-alpha)
    image = np.clip(image, 0, 255).astype('uint8')
    return image, mask

def augment_random_rotate(image, mask=None, limit=30):
    cols, rows = image.shape[:2]
    assert cols == rows
    size = cols

    rotate = np.random.randint(-1 * limit, limit)

    M = cv2.getRotationMatrix2D((int(size / 2), int(size / 2)), rotate, 1)
    image = cv2.warpAffine(image, M, (size, size))
    if mask is not None:
        mask = cv2.warpAffine(mask, M, (size, size))
    return image, mask

def augment_random_cover(image, mask=None, cover_ratio=0.2):
    cols, rows = image.shape[:2]
    cols == min([cols, rows])
    size = cols

    cover_size = max(int(size * cover_ratio), 1)
    if cover_size >= size:
        return image
    min_row = np.random.choice(size - cover_size)
    min_col = np.random.choice(size - cover_size)

    image[min_row:min_row+cover_size, min_col:min_col+cover_size] = 0
    if mask is not None:
        mask[min_row:min_row + cover_size, min_col:min_col + cover_size] = 0
    return image, mask

def augment_random_crop(image, mask=None, limit=0.10):

    H, W = image.shape[:2]

    dy = int(H*limit)
    y0 =   np.random.randint(0,dy)
    y1 = H-np.random.randint(0,dy)

    dx = int(W*limit)
    x0 =   np.random.randint(0,dx)
    x1 = W-np.random.randint(0,dx)

    image, mask = do_random_crop( image, mask, x0, y0, x1, y1 )
    return image, mask

def do_random_crop( image, mask=None, x0=0, y0=0, x1=1, y1=1 ):
    height, width = image.shape[:2]
    image = image[y0:y1,x0:x1]
    image = cv2.resize(image,dsize=(width,height), interpolation=cv2.INTER_LINEAR)

    if mask is not None:
        mask  = mask [y0:y1,x0:x1]
        mask  = cv2.resize(mask,dsize=(width,height), interpolation=cv2.INTER_LINEAR)

    return image, mask

def augment_random_Gaussian(image, mask=None, limit=0.3):
    sigma = np.random.uniform(0, limit)
    aug = iaa.GaussianBlur(sigma=sigma)
    image = aug.augment_images([image])[0]
    return image, mask

In [29]:
def train_multi_augment(image, mask):
    return image, mask 

###########################################################################################

def albu_augment_default(image, mask=None):
    return image, mask

def albu_augment_normalize(image, mask=None):
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    augment = A.Normalize(mean, std)
    image = augment(image=image)['image']
    
    return image, mask

In [30]:
class SchedulerBase(object):
    def __init__(self):
        self._is_load_best_weight = False
        self._is_load_best_optim = False
        self._is_freeze_bn=False
        self._is_adjust_lr = True
        self._lr = 0.01
        self._cur_optimizer = None

    def schedule(self,net, epoch, epochs, **kwargs):
        raise Exception('Did not implemented')

    def step(self, net, epoch, epochs):
        optimizer, lr = self.schedule(net, epoch, epochs)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        lr_list = []
        for param_group in optimizer.param_groups:
            lr_list += [param_group['lr']]
        return lr_list

    def is_load_best_weight(self):
        return self._is_load_best_weight

    def is_load_best_optim(self):
        return self._is_load_best_optim

    def is_freeze_bn(self):
        return self._is_freeze_bn

    def reset(self):
        self._is_load_best_weight = False
        self._load_best_optim = False
        self._is_freeze_bn = False

    def is_adjust_lr(self):
        return self._is_adjust_lr
    
class Adam3(SchedulerBase):
    def __init__(self,params_list=None):
        super(Adam3, self).__init__()
        self._lr = 1e-4
        self._cur_optimizer = None
        self.params_list=params_list
    def schedule(self,net, epoch, epochs, **kwargs):
        lr = 1e-4
        if epoch > 25:
            lr = 5e-5
        if epoch > 35:
            lr = 1e-5
        self._lr = lr
        if self._cur_optimizer is None:
            self._cur_optimizer = optim.Adam(net.parameters(), lr=lr)  # , weight_decay=0.0005
        return self._cur_optimizer, self._lr

In [32]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        # print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model


def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model

In [33]:
__all__ = ['SENet', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152',
           'se_resnext50_32x4d', 'se_resnext101_32x4d']

pretrained_settings = {
    'senet154': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/senet154-c7b49a05.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnet50': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnet50-ce0d4300.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnet101': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnet101-7e38fcc6.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnet152': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnet152-d17c99b7.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnext50_32x4d': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnext101_32x4d': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
}


class SEModule(nn.Module):

    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1,
                             padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1,
                             padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x


class Bottleneck(nn.Module):
    """
    Base class for bottlenecks that implements `forward()` method.
    """
    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = self.se_module(out) + residual
        out = self.relu(out)

        return out


class SEBottleneck(Bottleneck):
    """
    Bottleneck for SENet154.
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None):
        super(SEBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes * 2, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes * 2)
        self.conv2 = nn.Conv2d(planes * 2, planes * 4, kernel_size=3,
                               stride=stride, padding=1, groups=groups,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(planes * 4)
        self.conv3 = nn.Conv2d(planes * 4, planes * 4, kernel_size=1,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride


class SEResNetBottleneck(Bottleneck):
    """
    ResNet bottleneck with a Squeeze-and-Excitation module. It follows Caffe
    implementation and uses `stride=stride` in `conv1` and not in `conv2`
    (the latter is used in the torchvision implementation of ResNet).
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None):
        super(SEResNetBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False,
                               stride=stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1,
                               groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride


class SEResNeXtBottleneck(Bottleneck):
    """
    ResNeXt bottleneck type C with a Squeeze-and-Excitation module.
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None, base_width=4):
        super(SEResNeXtBottleneck, self).__init__()
        width = math.floor(planes * (base_width / 64)) * groups
        self.conv1 = nn.Conv2d(inplanes, width, kernel_size=1, bias=False,
                               stride=1)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride,
                               padding=1, groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride


class SENet(nn.Module):

    def __init__(self, block, layers, groups, reduction, dropout_p=0.2,
                 inplanes=128, input_3x3=True, downsample_kernel_size=3,
                 downsample_padding=1, num_classes=1000):
        """
        Parameters
        ----------
        block (nn.Module): Bottleneck class.
            - For SENet154: SEBottleneck
            - For SE-ResNet models: SEResNetBottleneck
            - For SE-ResNeXt models:  SEResNeXtBottleneck
        layers (list of ints): Number of residual blocks for 4 layers of the
            network (layer1...layer4).
        groups (int): Number of groups for the 3x3 convolution in each
            bottleneck block.
            - For SENet154: 64
            - For SE-ResNet models: 1
            - For SE-ResNeXt models:  32
        reduction (int): Reduction ratio for Squeeze-and-Excitation modules.
            - For all models: 16
        dropout_p (float or None): Drop probability for the Dropout layer.
            If `None` the Dropout layer is not used.
            - For SENet154: 0.2
            - For SE-ResNet models: None
            - For SE-ResNeXt models: None
        inplanes (int):  Number of input channels for layer1.
            - For SENet154: 128
            - For SE-ResNet models: 64
            - For SE-ResNeXt models: 64
        input_3x3 (bool): If `True`, use three 3x3 convolutions instead of
            a single 7x7 convolution in layer0.
            - For SENet154: True
            - For SE-ResNet models: False
            - For SE-ResNeXt models: False
        downsample_kernel_size (int): Kernel size for downsampling convolutions
            in layer2, layer3 and layer4.
            - For SENet154: 3
            - For SE-ResNet models: 1
            - For SE-ResNeXt models: 1
        downsample_padding (int): Padding for downsampling convolutions in
            layer2, layer3 and layer4.
            - For SENet154: 1
            - For SE-ResNet models: 0
            - For SE-ResNeXt models: 0
        num_classes (int): Number of outputs in `last_linear` layer.
            - For all models: 1000
        """
        super(SENet, self).__init__()
        self.inplanes = inplanes
        if input_3x3:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, 64, 3, stride=2, padding=1,
                                    bias=False)),
                ('bn1', nn.BatchNorm2d(64)),
                ('relu1', nn.ReLU(inplace=True)),
                ('conv2', nn.Conv2d(64, 64, 3, stride=1, padding=1,
                                    bias=False)),
                ('bn2', nn.BatchNorm2d(64)),
                ('relu2', nn.ReLU(inplace=True)),
                ('conv3', nn.Conv2d(64, inplanes, 3, stride=1, padding=1,
                                    bias=False)),
                ('bn3', nn.BatchNorm2d(inplanes)),
                ('relu3', nn.ReLU(inplace=True)),
            ]
        else:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, inplanes, kernel_size=7, stride=2,
                                    padding=3, bias=False)),
                ('bn1', nn.BatchNorm2d(inplanes)),
                ('relu1', nn.ReLU(inplace=True)),
            ]
        # To preserve compatibility with Caffe weights `ceil_mode=True`
        # is used instead of `padding=1`.
        layer0_modules.append(('pool', nn.MaxPool2d(3, stride=2,
                                                    ceil_mode=True)))
        self.layer0 = nn.Sequential(OrderedDict(layer0_modules))
        self.layer1 = self._make_layer(
            block,
            planes=64,
            blocks=layers[0],
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=1,
            downsample_padding=0
        )
        self.layer2 = self._make_layer(
            block,
            planes=128,
            blocks=layers[1],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer3 = self._make_layer(
            block,
            planes=256,
            blocks=layers[2],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer4 = self._make_layer(
            block,
            planes=512,
            blocks=layers[3],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_p) if dropout_p is not None else None
        self.last_linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, groups, reduction, stride=1,
                    downsample_kernel_size=1, downsample_padding=0):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=downsample_kernel_size, stride=stride,
                          padding=downsample_padding, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, groups, reduction, stride,
                            downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups, reduction))

        return nn.Sequential(*layers)

    def features(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def logits(self, x):
        x = self.avg_pool(x)
        if self.dropout is not None:
            x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, x):
        x = self.features(x)
        x = self.logits(x)
        return x


def senet154(num_classes=1000, pretrained='imagenet'):
    model = SENet(SEBottleneck, [3, 8, 36, 3], groups=64, reduction=16,
                  dropout_p=0.2, num_classes=num_classes)
    return model


def se_resnet50(num_classes=1000):
    model = SENet(SEResNetBottleneck, [3, 4, 6, 3], groups=1, reduction=16,
                  dropout_p=None, inplanes=64, input_3x3=False,
                  downsample_kernel_size=1, downsample_padding=0,
                  num_classes=num_classes)
    return model


def se_resnet101(num_classes=1000):
    model = SENet(SEResNetBottleneck, [3, 4, 23, 3], groups=1, reduction=16,
                  dropout_p=None, inplanes=64, input_3x3=False,
                  downsample_kernel_size=1, downsample_padding=0,
                  num_classes=num_classes)
    return model


def se_resnet152(num_classes=1000):
    model = SENet(SEResNetBottleneck, [3, 8, 36, 3], groups=1, reduction=16,
                  dropout_p=None, inplanes=64, input_3x3=False,
                  downsample_kernel_size=1, downsample_padding=0,
                  num_classes=num_classes)

    return model


def se_resnext50_32x4d(num_classes=1000):
    model = SENet(SEResNeXtBottleneck, [3, 4, 6, 3], groups=32, reduction=16,
                  dropout_p=None, inplanes=64, input_3x3=False,
                  downsample_kernel_size=1, downsample_padding=0,
                  num_classes=num_classes)
    return model


def se_resnext101_32x4d(num_classes=1000):
    model = SENet(SEResNeXtBottleneck, [3, 4, 23, 3], groups=32, reduction=16,
                  dropout_p=None, inplanes=64, input_3x3=False,
                  downsample_kernel_size=1, downsample_padding=0,
                  num_classes=num_classes)
    return model

In [34]:
class CBAM_Module(nn.Module):
    def __init__(self, channels, reduction,attention_kernel_size=3):
        super(CBAM_Module, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1,
                             padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1,
                             padding=0)
        self.sigmoid_channel = nn.Sigmoid()
        k=2
        self.conv_after_concat = nn.Conv2d(k, 1,
                                           kernel_size = attention_kernel_size,
                                           stride=1,
                                           padding = attention_kernel_size//2)
        self.sigmoid_spatial = nn.Sigmoid()

    def forward(self, x):
        # Channel attention module
        module_input = x
        avg = self.avg_pool(x)
        mx = self.max_pool(x)
        avg = self.fc1(avg)
        mx = self.fc1(mx)
        avg = self.relu(avg)
        mx = self.relu(mx)
        avg = self.fc2(avg)
        mx = self.fc2(mx)
        x = avg + mx
        x = self.sigmoid_channel(x)
        # Spatial attention module
        x = module_input * x
        module_input = x
        b, c, h, w = x.size()
        avg = torch.mean(x, 1, True)
        mx, _ = torch.max(x, 1, True)
        x = torch.cat((avg, mx), 1)
        x = self.conv_after_concat(x)
        x = self.sigmoid_spatial(x)
        x = module_input * x
        return x


class ConvBn2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3,3), stride=(1,1), padding=(1,1)):
        super(ConvBn2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
    def forward(self, z):
        x = self.conv(z)
        x = self.bn(x)
        return x


class ConvBnRelu2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3,3), stride=(1,1), padding=(1,1)):
        super(ConvBnRelu2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
    def forward(self, z):
        x = self.conv(z)
        x = self.bn(x)
        x = F.relu(x, inplace=True)
        return x


class Decoder(nn.Module):
    def __init__(self, in_channels, channels, out_channels ,
                 up_sample=True,
                 attention_type=None,
                 attention_kernel_size=3,
                 reduction=16,
                 reslink=False):
        super(Decoder, self).__init__()
        self.conv1 = ConvBn2d(in_channels,  channels, kernel_size=3, padding=1)
        self.conv2 = ConvBn2d(channels, out_channels, kernel_size=3, padding=1)
        self.up_sample = up_sample
        self.attention = attention_type is not None
        self.attention_type = attention_type
        self.reslink = reslink
        if attention_type is None:
            pass
        elif attention_type.find('cbam') >= 0:
            self.channel_gate = CBAM_Module(out_channels, reduction,attention_kernel_size)
        if self.reslink:
            self.shortcut = ConvBn2d(in_channels, out_channels, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0))
    def forward(self, x, size=None):
        if self.up_sample:
            if size is None:
                x = F.upsample(x, scale_factor=2, mode='bilinear', align_corners=True)  # False
            else:
                x = F.upsample(x, size=size, mode='bilinear', align_corners=True)
        if self.reslink:
            shortcut = self.shortcut(x)
        x = F.relu(self.conv1(x), inplace=True)
        x = F.relu(self.conv2(x), inplace=True)
        if self.attention:
            x = self.channel_gate(x)
        if self.reslink:
            x = F.relu(x+shortcut)
        return x

In [35]:
class ResnetUnet(nn.Module):

    def load_pretrain(self, pretrain_file):
        print('load pretrained file: %s' % pretrain_file)
        self.resnet.load_state_dict(torch.load(pretrain_file, map_location=lambda storage, loc: storage))

    def __init__(self,feature_net='resnet34',
                 attention_type=None,
                 reduction=16,
                 reslink=False,
                 out_channel=4,
                 pretrained_file=None,
                 ):
        super().__init__()
        self.attention = attention_type is not None
        self.attention_type = attention_type
        self.out_channel = out_channel
        decoder_kernels = [1, 1, 1, 1, 1]
        if feature_net == 'resnet18':
            self.resnet = resnet18()
            self.EX = 1
        elif feature_net == 'resnet34':
            self.resnet = resnet34()
            self.EX = 1
        elif feature_net == 'resnet50':
            self.resnet = resnet50()
            self.EX = 4
        elif feature_net == 'resnet101':
            self.resnet = resnet101()
            self.EX = 4
        elif feature_net == 'resnet152':
            self.resnet = resnet152()
            self.EX = 4

        self.load_pretrain(pretrained_file)
        self.conv1 = nn.Sequential(
            self.resnet.conv1,
            self.resnet.bn1,
            self.resnet.relu,
        )
        self.encoder2 = self.resnet.layer1
        self.encoder3 = self.resnet.layer2
        self.encoder4 = self.resnet.layer3
        self.encoder5 = self.resnet.layer4
        att_type=self.attention_type
        self.decoder4 = Decoder(256*self.EX + 32, 256, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[1],
                                reduction=reduction,
                                reslink=reslink)
        self.decoder3 = Decoder(128*self.EX + 32, 128, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[2],
                                reduction=reduction,
                                reslink=reslink)
        self.decoder2 = Decoder(64*self.EX + 32, 64, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[3],
                                reduction=reduction,
                                reslink=reslink)
        self.decoder1 = Decoder(32, 32, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[4],
                                reduction=reduction,
                                reslink=reslink)

        self.logit = nn.Sequential(
            ConvBnRelu2d(160, 64, kernel_size=3, padding=1),
            ConvBnRelu2d(64, 32, kernel_size=3, padding=1),
            nn.Conv2d(32, out_channel, kernel_size=1, padding=0),
        )

        center_channels = 512 * self.EX
        decoder5_channels = 512 * self.EX + 256 * self.EX

        self.center = nn.Sequential(
            ConvBn2d(center_channels, 512 * self.EX, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            ConvBn2d(512 * self.EX, 256 * self.EX, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.decoder5 = Decoder(decoder5_channels, 512, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[0],
                                reduction=reduction,
                                reslink=reslink)

    def forward(self, x, *args):
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        e2 = self.encoder2(x)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        e5 = self.encoder5(e4)

        f = self.center(e5)

        d5 = self.decoder5(torch.cat([f, e5], 1))
        d4 = self.decoder4(torch.cat([d5, e4], 1))
        d3 = self.decoder3(torch.cat([d4, e3], 1))
        d2 = self.decoder2(torch.cat([d3, e2], 1))
        d1 = self.decoder1(d2)
        f = torch.cat((
                 d1,
                 F.upsample(d2, scale_factor=2, mode='bilinear',align_corners=False),
                 F.upsample(d3, scale_factor=4, mode='bilinear', align_corners=False),
                 F.upsample(d4, scale_factor=8, mode='bilinear', align_corners=False),
                 F.upsample(d5, scale_factor=16, mode='bilinear', align_corners=False),
        ), 1)
        logit = self.logit(f)
        return logit

def unet_resnet34_cbam_v0a(**kwargs):
    pretrained_file = kwargs['pretrained_file']
    model = ResnetUnet(feature_net='resnet34', attention_type='cbam_v0a', pretrained_file=pretrained_file)
    return model

In [36]:
class SEnetUnet(nn.Module):
    def load_pretrain(self, pretrain_file):
        print('load pretrained file: %s' % pretrain_file)
        self.backbone.load_state_dict(torch.load(pretrain_file, map_location=lambda storage, loc: storage))
    #attention_type
    #none:no attention
    #scse_v0:1803.02579 Concurrent Spatial and Channel  Squeeze & Excitation in Fully Convolutional Networks.pdf
    #https://github.com/Youngkl0726/Convolutional-Block-Attention-Module/blob/master/CBAMNet.py
    #https://github.com/kobiso/CBAM-keras/blob/master/models/attention_module.py
    def __init__(self,
                 feature_net='se_resnext50_32x4d',
                 attention_type=None,
                 reduction=16,
                 pretrained_file=None,
                 ):
        super().__init__()
        self.attention = attention_type is not None
        self.attention_type = attention_type
        decoder_kernels = [1, 1, 1, 1, 1]
        if feature_net == 'se_resnext50_32x4d':
            self.backbone = se_resnext50_32x4d()
            self.EX = 4
        if feature_net == 'se_resnext101_32x4d':
            self.backbone = se_resnext101_32x4d()
            self.EX = 4
        elif feature_net == 'se_resnet50':
            self.backbone = se_resnet50()
            self.EX = 4
        elif feature_net == 'se_resnet101':
            self.backbone = se_resnet101()
            self.EX = 4
        elif feature_net == 'se_resnet152':
            self.backbone = se_resnet152()
            self.EX = 4
        elif feature_net == 'senet154':
            self.backbone = senet154()
            self.EX = 4

        self.load_pretrain(pretrained_file)
        self.conv1 =nn.Sequential(*list(self.backbone.layer0.children())[:-1])
        self.encoder2 = self.backbone.layer1  # 64*self.EX
        self.encoder3 = self.backbone.layer2  # 128*self.EX
        self.encoder4 = self.backbone.layer3  # 256*self.EX
        self.encoder5 = self.backbone.layer4  # 512*self.EX
        self.center = nn.Sequential(
            ConvBn2d(512*self.EX, 512*self.EX, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            ConvBn2d(512*self.EX, 256*self.EX, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        att_type=self.attention_type
        self.decoder5 = Decoder(512*self.EX + 256*self.EX, 512, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[0],
                                reduction=reduction)
        self.decoder4 = Decoder(256*self.EX + 32, 256, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[1],
                                reduction=reduction)
        self.decoder3 = Decoder(128*self.EX + 32, 128, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[2],
                                reduction=reduction)
        self.decoder2 = Decoder(64*self.EX + 32, 64, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[3],
                                reduction=reduction)
        self.decoder1 = Decoder(32, 32, 32,
                                attention_type=att_type,
                                attention_kernel_size=decoder_kernels[4],
                                reduction=reduction)
        self.logit = nn.Sequential(
            ConvBnRelu2d(160, 64, kernel_size=3, padding=1),
            ConvBnRelu2d(64, 32, kernel_size=3, padding=1),
            nn.Conv2d(32, 4, kernel_size=1, padding=0),
        )
    def forward(self, x, *args):
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        e2 = self.encoder2(x)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        e5 = self.encoder5(e4)

        f = self.center(e5)
        d5 = self.decoder5(torch.cat([f, e5], 1))
        d4 = self.decoder4(torch.cat([d5, e4], 1))
        d3 = self.decoder3(torch.cat([d4, e3], 1))
        d2 = self.decoder2(torch.cat([d3, e2], 1))
        d1 = self.decoder1(d2)

        f = torch.cat((
                 d1,
                 F.upsample(d2, scale_factor=2, mode='bilinear',align_corners=False),
                 F.upsample(d3, scale_factor=4, mode='bilinear', align_corners=False),
                 F.upsample(d4, scale_factor=8, mode='bilinear', align_corners=False),
                 F.upsample(d5, scale_factor=16, mode='bilinear', align_corners=False),

        ), 1)

        logit = self.logit(f)
        return logit

def unet_se_resnext50_cbam_v0a(**kwargs):
    pretrained_file = kwargs['pretrained_file']
    model = SEnetUnet(feature_net='se_resnext50_32x4d', attention_type='cbam_v0a', pretrained_file=pretrained_file)
    return model

In [37]:
def run_length_encode(component):
    if component.sum() == 0:
        return ''
    component = np.hstack([np.array([0]), component.T.flatten(), np.array([0])])
    start  = np.where(component[1: ] > component[:-1])[0]
    end    = np.where(component[:-1] > component[1: ])[0]
    length = end-start
      
    rle = []
    for i in range(len(length)):
        if i==0:
            rle.extend([start[0], length[0]])
        else:
            rle.extend([start[i], length[i]])

    rle = ' '.join([str(r) for r in rle])
    return rle

def run_length_decode(rle, height=256, width=1600, fill_value=1.):
    component = np.zeros((height, width), np.float32)
    if str(rle) == 'nan' or rle == 0:
        return component
    component = component.reshape(-1)
    rle  = np.array([int(s) for s in rle.split(' ')])
    rle  = rle.reshape(-1, 2)
    start = 0
    for index,length in rle:
        start = index
        end   = start+length
        component[start : end] = fill_value
        start = end

    component = component.reshape(width, height).T
    return component

def build_mask(s, height, width):
    mask = np.zeros((height, width, 4))
    for i in range(4):
        mask[:,:,i] = run_length_decode(s[f'{i+1}'], height, width)
    return mask

In [38]:
model_names = {
    'unet_resnet34_cbam_v0a': 'resnet34-333f7ec4.pth',
    'unet_se_resnext50_cbam_v0a': 'se_resnext50_32x4d-a260b3a4.pth',
}

def init_network(params):
    architecture = params.get('architecture', 'unet_resnet34_cbam_v0a')
    pretrained_file = opj(PRETRAINED_DIR, model_names[architecture])
    print(">> Using pre-trained model.")
    net = eval(architecture)(pretrained_file=pretrained_file)
    return net

In [39]:
def image_to_tensor(image, mean=0, std=1.):
    image = image.astype(np.float32)
    image = (image-mean)/std
    if len(image.shape) == 3:
        image = image.transpose((2,0,1))
    tensor = torch.from_numpy(image)

    return tensor

def label_to_tensor(label):
    label_ret = label.astype(np.float32)
    label_ret = (label_ret > 0.1).astype(np.float32)
    if len(label_ret.shape) == 3:
        label_ret = label_ret.transpose((2,0,1)) 
    tensor = torch.from_numpy(label_ret).type(torch.FloatTensor)
    return tensor

In [40]:
class SteelDataset(Dataset):
    def __init__(self, 
                 steel_df,
                 img_size=(256, 1600),
                 mask_size=(256, 1600),
                 transform=None,
                 return_label=True,
                 pseudo=None,
                 pseudo_ratio=0.,
                 crop_version=None,
                 dataset=None,
                 predict_pos=False):
        self.img_size = img_size
        self.mask_size = mask_size
        self.return_label = return_label
        self.crop_version = crop_version
        self.dataset = dataset
        self.suffix = 'jpg'
       
        base_dir = DATA_DIR
        if dataset in ['train', 'val']:
            if crop_version is None:
                img_dir = opj(base_dir, 'train', 'images', 'images_256x1600')
                print(img_dir)
        elif dataset in ['test']:
            if crop_version is None:
                img_dir = opj(base_dir, 'test', 'images', 'images_256x1600')
        else:
            raise ValueError(dataset)

        if predict_pos:
            steel_df = steel_df[steel_df[STEEL] == 1]
        steel_df = steel_df.fillna(0)
        steel_df['pseudo'] = False
        self.steel_df = steel_df

        if crop_version is None:
            self.img_ids = self.steel_df[ID].values
        else:
            self.img_ids = self.steel_df[CROP_ID].values

        self.img_dir = img_dir
        self.num = len(self.img_ids)

        self.basic_img_ids = self.img_ids
        self.transform = transform

        # self.pseudo_flag = self.steel_df['pseudo'].values
        # self.basic_pseudo_flag = self.pseudo_flag
        # self.pseudo = pseudo
        # self.pseudo_ratio = pseudo_ratio

        if (dataset == 'train') and return_label:
            if 'pos' in self.steel_df.columns:
                self.pos_flag = self.steel_df['pos']
                self.pos_steel_df = self.steel_df[self.pos_flag]
                self.neg_steel_df = self.steel_df[~self.pos_flag]
            else:
                self.pos_flag = self.steel_df[SPLIT] != 0
                self.pos_steel_df = self.steel_df[self.pos_flag]
                self.neg_steel_df = self.steel_df[~self.pos_flag]
        else:
            self.pos_steel_df = None
            self.neg_steel_df = None

        # if pseudo is not None:
        #     self.pseudo_list = pseudo.split(',')
        #     self.pos_pseudo_df_list = []
        #     self.neg_pseudo_df_list = []
        #     for pseudo in self.pseudo_list:
        #         pseudo_df = pd.read_csv(opj(DATA_DIR, 'pseudo', '%s.csv' % pseudo))
        #         pseudo_df['pseudo'] = True
        #         if CROP_ID not in pseudo_df.columns:
        #             pseudo_df[CROP_ID] = pseudo_df[ID]
        #         if 'chexpert' in pseudo:
        #             pseudo_df['suffix'] = 'jpg'
        #         else:
        #             pseudo_df['suffix'] = 'png'
        #         pseudo_df['pseudo_name'] = pseudo
        #         pos_pseudo_df = pseudo_df[pseudo_df[SPLIT] != '-1']
        #         neg_pseudo_df = pseudo_df[pseudo_df[SPLIT] == '-1']

        #         self.pos_pseudo_df_list.append(pos_pseudo_df)
        #         self.neg_pseudo_df_list.append(neg_pseudo_df)
        #     self.resample_pseudo(first=True)

        print('image_dir: %s' % self.img_dir)
        print('image size: %s' % str(self.img_size))

    def resample_pseudo(self, first=False):
        steel_df_list = [self.pos_steel_df, self.neg_steel_df]
        if first:
            print('%s pos_num: %d neg_num: %d' % ('train', len(steel_df_list[-2]), len(steel_df_list[-1])))

        for idx, (pos_pseudo_df, neg_pseudo_df) in enumerate(zip(self.pos_pseudo_df_list, self.neg_pseudo_df_list)):
            pos_pseudo_size = int(min(len(self.pos_steel_df) * self.pseudo_ratio, len(pos_pseudo_df)))
            neg_pseudo_size = int(min(len(self.neg_steel_df) / len(self.pos_steel_df) * pos_pseudo_size, len(neg_pseudo_df)))

            pos_pseudo_size = min(len(pos_pseudo_df), pos_pseudo_size)
            neg_pseudo_size = min(len(neg_pseudo_df), neg_pseudo_size)
            steel_df_list.append(pos_pseudo_df.iloc[np.random.choice(len(pos_pseudo_df), size=pos_pseudo_size, replace=False)])
            steel_df_list.append(neg_pseudo_df.iloc[np.random.choice(len(neg_pseudo_df), size=neg_pseudo_size, replace=False)])

            if first:
                if self.crop_version is None:
                    print('pseudo dir: %s' % opj(DATA_DIR, pos_pseudo_df.iloc[0][DATASET],
                                                 'images', 'images_%d' % self.img_size))
                else:
                    print('pseudo dir: %s' % opj(DATA_DIR, pos_pseudo_df.iloc[0][DATASET],
                                                 'images', 'images_%s_%d' % (self.crop_version, self.img_size)))
                print('%s pos_num: %d neg_num: %d' % (self.pseudo_list[idx], len(steel_df_list[-2]), len(steel_df_list[-1])))

        steel_df = pd.concat(steel_df_list)
        if self.crop_version is None:
            self.img_ids = steel_df[ID].values
        else:
            self.img_ids = steel_df[CROP_ID].values
        self.pseudo_flag = steel_df['pseudo'].values
        self.dataset = steel_df[DATASET].values
        self.pos_flag = steel_df[SPLIT] != 0
        self.pseudo_suffix = steel_df['suffix'].values
        self.pseudo_name = steel_df['pseudo_name'].values
        self.num = len(self.img_ids)

    def resample(self):
        pass

    def __getitem__(self, index):
        # is_pseudo = self.pseudo_flag[index]
        img_id = self.img_ids[index]

        # if is_pseudo:
        #     dataset = self.dataset[index]
        #     pseudo_suffix = self.pseudo_suffix[index]
        #     if self.crop_version is None:
        #         img_fname = opj(DATA_DIR, dataset, 'images', 'images_1024',
        #                         '%s.%s' % (img_id, pseudo_suffix))
        #     else:
        #         img_fname = opj(DATA_DIR, dataset, 'images', 'images_%s' % self.crop_version,
        #                         '%s.%s' % (img_id, pseudo_suffix))
        # else:
        #     img_fname = opj(self.img_dir, '%s.%s' % (img_id, self.suffix))
        img_fname = opj(self.img_dir, f'{img_id}')

        image = cv2.imread(img_fname)
        if image is None:
            print(img_fname)
        #if image.shape[0] != self.img_size[0] or image.shape[1] != self.img_size[1]:
        #    image = cv2.resize(image, (self.img_size[0], self.img_size[1]), interpolation=cv2.INTER_LINEAR)

        if self.return_label:
            # if is_pseudo:
            #     dataset = self.dataset[index]
            #     pseudo_name = self.pseudo_name[index]
            #     if self.crop_version is None:
            #         mask_file = opj(DATA_DIR, dataset, 'images', 'masks_1024', '%s.png' % img_id)
            #     else:
            #         if pseudo_name is not None and '8740_pseudo' in pseudo_name:
            #             mask_file = opj(DATA_DIR, dataset, 'images', 'masks_8740_%s' % self.crop_version,
            #                             '%s.png' % img_id)
            #         else:
            #             mask_file = opj(DATA_DIR, dataset, 'images', 'masks_%s' % self.crop_version,
            #                             '%s.png' % img_id)
            # else:
            #     mask_file = opj(self.mask_dir, '%s.png' % img_id)
            
            mask = build_mask(self.steel_df.iloc[index], self.mask_size[0], self.mask_size[1])
            mask = mask.astype(np.int8)

            if self.transform is not None:
                image, mask = self.transform(image=image, mask=mask)

            image = image / 255.0
            # mask = mask / 255.0
            image = image_to_tensor(image)
            mask = label_to_tensor(mask)

            return image, mask, index
        else:
            if self.transform is not None:
                image = self.transform(image=image)[0]

            image = image / 255.0

            image = image_to_tensor(image)
            return image, index

    def __len__(self):
        return self.num


# https://www.kaggle.com/c/siim-acr-STEEL-segmentation/discussion/97456
class BalanceClassSampler(Sampler):
    def __init__(self, dataset, length=None):
        self.dataset = dataset
        if length is None:
            length = len(self.dataset)
        self.length = int(length)

        half = self.length // 2 + 1
        self.pos_length = half
        self.neg_length = half
        print('pos num: %s, neg num: %s' % (self.pos_length, self.neg_length))

    def __iter__(self):
        pos_index = np.where(self.dataset.pos_flag)[0]
        neg_index = np.where(~self.dataset.pos_flag)[0]

        pos = np.random.choice(pos_index, self.pos_length, replace=True)
        neg = np.random.choice(neg_index, self.neg_length, replace=True)

        l = np.hstack([pos, neg]).T
        l = l.reshape(-1)
        np.random.shuffle(l)
        l = l[:self.length]
        return iter(l)

    def __len__(self):
        return self.length

In [41]:
"""
Lovasz-Softmax and Jaccard hinge loss in PyTorch
Maxim Berman 2018 ESAT-PSI KU Leuven (MIT License)
"""

from __future__ import print_function, division
try:
    from itertools import  ifilterfalse
except ImportError: # py3k
    from itertools import  filterfalse

def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts - gt_sorted.float().cumsum(0)
    union = gts + (1 - gt_sorted).float().cumsum(0)
    jaccard = 1. - intersection / union
    if p > 1: # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard

def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        loss = mean(lovasz_hinge_flat(*flatten_binary_scores(log.unsqueeze(0), lab.unsqueeze(0), ignore))
                    for log, lab in zip(logits, labels))
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss

def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels.float() - 1.
    errors = (1. - logits * Variable(signs))
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad = lovasz_grad(gt_sorted)
    # loss = torch.dot(F.elu(errors_sorted)+1, Variable(grad))
    loss = torch.dot(F.relu(errors_sorted), Variable(grad))
    return loss

def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = scores.view(-1)
    labels = labels.view(-1)
    if ignore is None:
        return scores, labels
    valid = (labels != ignore)
    vscores = scores[valid]
    vlabels = labels[valid]
    return vscores, vlabels

def mean(l, ignore_nan=False, empty=0):
    """
    nanmean compatible with generators.
    """
    l = iter(l)
    if ignore_nan:
        l = ifilterfalse(np.isnan, l)
    try:
        n = 1
        acc = next(l)
    except StopIteration:
        if empty == 'raise':
            raise ValueError('Empty mean')
        return empty
    for n, v in enumerate(l, 2):
        acc += v
    if n == 1:
        return acc
    return acc / n

In [42]:
def dice_score(prob, truth, threshold=0.5):
    num = prob.size(0)

    prob = prob > threshold
    truth = truth > 0.5

    prob = prob.view(num, -1)
    truth = truth.view(num, -1)
    intersection = (prob * truth)

    score = 2. * (intersection.sum(1) + 1.).float() / (prob.sum(1) + truth.sum(1) + 2.).float()
    score[score >= 1] = 1
    score = score.sum() / num
    return score


class SymmetricLovaszLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SymmetricLovaszLoss, self).__init__()
    
    def forward(self, logits, targets):
        return ((L.lovasz_hinge(logits, targets, per_image=True)) + (L.lovasz_hinge(-logits, 1-targets, per_image=True))) / 2

In [43]:
def do_kaggle_metric(predict, truth, threshold=0.5):
    num = len(predict)

    prob = predict > threshold
    truth = truth > 0.5

    prob = prob.reshape(num, -1)
    truth = truth.reshape(num, -1)
    intersection = (prob * truth)

    score = 2. * (intersection.sum(1) + EPS) / (prob.sum(1) + truth.sum(1) + EPS)
    score[score >= 1] = 1
    return score

def get_batch_kaggle_score(param):
    df_submit, ids, start, batch_size = param
    end = np.minimum(start + batch_size, len(ids))
    N = end - start
    predict = np.zeros((N,IMG_SIZE,IMG_SIZE),np.bool)
    truth   = np.zeros((N,IMG_SIZE,IMG_SIZE),np.bool)
    for i,n in enumerate(range(start, end)):
        id = ids[n]
        p = df_submit.loc[id][TARGET+'_pred']
        t = df_submit.loc[id][TARGET]
        p = run_length_decode(p,fill_value=1)
        t = run_length_decode(t,fill_value=1)
        predict[i] = p
        truth[i] = t
    score = do_kaggle_metric(predict, truth, threshold=0.5)
    return score

def get_kaggle_score(df_submit, df_truth=None, result_type='val', pool=None, return_mean=True):
    if df_truth is None:
        if result_type == 'val':
            df_truth = pd.read_csv(opj(DATA_DIR, 'meta', 'train-rle.csv'))
        else:
            return 0.

    df_submit = df_submit.merge(df_truth, how='left', on=ID, suffixes=('_pred', ''))
    ids = df_submit[ID].values
    df_submit = df_submit.set_index(ID).fillna('-1')

    batch_size = 100000
    N = len(ids)
    iter_count = int(np.ceil(N / batch_size))
    params = []
    for it in range(iter_count):
        start = it * batch_size
        params.append((df_submit, ids, start, batch_size))

    score = get_batch_kaggle_score(params[0])
    if return_mean:
        score_mean = np.mean(score)
        return score_mean
    else:
        return score, ids

def get_kaggle_score_prob(prob_submit, image_ids):

    df_truth = pd.read_csv(opj(DATA_DIR, 'meta', 'train-rle.csv'))
    ids_submit = pd.DataFrame(image_ids, columns=[ID])
    ids_submit = ids_submit.merge(df_truth, how='left', on=ID)
    ids_submit = ids_submit.set_index(ID).fillna('-1')
    prob_truth = np.zeros((len(ids_submit), IMG_SIZE, IMG_SIZE), np.bool)
    for i, t in enumerate(ids_submit[TARGET].values):
        t = run_length_decode(t, fill_value=1)
        prob_truth[i] = t
    score = np.mean(do_kaggle_metric(prob_submit, prob_truth))
    return score

In [44]:
class Logger(object):
    def __init__(self):
        self.terminal = sys.stdout
        self.file = None

    def open(self, file, mode=None):
        if mode is None: mode ='w'
        self.file = open(file, mode)

    def write(self, message, is_terminal=1, is_file=1):
        if '\r' in message: is_file = 0

        if is_terminal == 1:
            self.terminal.write(message)
            self.terminal.flush()

        if is_file == 1:
            self.file.write(message)
            self.file.flush()

    def flush(self):       
        pass

In [45]:
def main():

    log_out_dir = opj(RESULT_DIR, 'logs', config['out_dir'])
    if not ope(log_out_dir):
        os.makedirs(log_out_dir)
    log = Logger()
    log.open(opj(log_out_dir, 'log.train.txt'), mode='a')

    model_out_dir = opj(RESULT_DIR, 'models', config['out_dir'])
    log.write(">> Creating directory if it does not exist:\n>> '{}'\n".format(model_out_dir))
    if not ope(model_out_dir):
        os.makedirs(model_out_dir)

    # set cuda visible device
    os.environ['CUDA_VISIBLE_DEVICES'] = config['gpu_id']
    cudnn.benchmark = True

    # set random seeds
    torch.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)

    model_params = {}
    model_params['architecture'] = config['arch']
    model = init_network(model_params)

    # move network to gpu
    model = DataParallel(model)
    model.cuda()

    if config['ema']:
        ema_model = copy.deepcopy(model)
        ema_model.cuda()
    else:
        ema_model = None

    # define loss function (criterion)
    try:
        criterion = eval(config['loss'])().cuda()
    except:
        raise(RuntimeError("Loss {} not available!".format(config['loss'])))

    start_epoch = 0
    best_epoch = 0
    best_dice = 0
    best_dice_arr = np.zeros(3)

    # define scheduler
    try:
        scheduler = eval(config['scheduler'])()
    except:
        raise (RuntimeError("Scheduler {} not available!".format(config['scheduler'])))
    optimizer = scheduler.schedule(model, start_epoch, config['epochs'])[0]

    # optionally resume from a checkpoint
    if config['resume']:
        model_fpath = os.path.join(model_out_dir, config['resume'])
        if os.path.isfile(model_fpath):
            # load checkpoint weights and update model and optimizer
            log.write(">> Loading checkpoint:\n>> '{}'\n".format(model_fpath))

            checkpoint = torch.load(model_fpath)
            start_epoch = checkpoint['epoch']
            best_epoch = checkpoint['best_epoch']
            best_dice_arr = checkpoint['best_dice_arr']
            best_dice = np.max(best_dice_arr)
            model.module.load_state_dict(checkpoint['state_dict'])

            optimizer_fpath = model_fpath.replace('.pth', '_optim.pth')
            if ope(optimizer_fpath):
                log.write(">> Loading checkpoint:\n>> '{}'\n".format(optimizer_fpath))
                optimizer.load_state_dict(torch.load(optimizer_fpath)['optimizer'])

            if config['ema']:
                ema_model_fpath = model_fpath.replace('.pth', '_ema.pth')
                if ope(ema_model_fpath):
                    log.write(">> Loading checkpoint:\n>> '{}'\n".format(ema_model_fpath))
                    ema_model.module.load_state_dict(torch.load(ema_model_fpath)['state_dict'])
            log.write(">>>> loaded checkpoint:\n>>>> '{}' (epoch {})\n".format(model_fpath, checkpoint['epoch']))
        else:
            log.write(">> No checkpoint found at '{}'\n".format(model_fpath))

    # Data loading code
    train_transform = eval(config['train_transform'])
    steel_df = pd.read_csv(opj('..', 'input', 'preprocessed_train.csv'))
    train_idx, valid_idx, _, _ = train_test_split(
                                            steel_df.index, 
                                            steel_df['split_label'], 
                                            test_size=0.2, 
                                            random_state=43)

    train_dataset = SteelDataset(
        steel_df.iloc[train_idx],
        img_size=config['img_size'],
        mask_size=config['img_size'],
        transform=train_transform,
        return_label=True,
        crop_version=config['crop_version'],
        pseudo=config['pseudo'],
        pseudo_ratio=config['pseudo_ratio'],
        dataset='train',
    )
    if config['is_balance']:
        train_sampler = BalanceClassSampler(train_dataset, config['sample_times'] * len(train_dataset))
    else:
        train_sampler = RandomSampler(train_dataset)
    train_loader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=config['batch_size'],
        drop_last=True,
        num_workers=config['workers'],
        pin_memory=True,
    )

    valid_dataset = SteelDataset(
        steel_df.iloc[valid_idx],
        img_size=config['img_size'],
        mask_size=config['img_size'],
        transform=None,
        return_label=True,
        crop_version=config['crop_version'],
        dataset='val',
    )
    valid_loader = DataLoader(
        valid_dataset,
        sampler=SequentialSampler(valid_dataset),
        batch_size=max(int(config['batch_size'] // 2), 1),
        drop_last=False,
        num_workers=config['workers'],
        pin_memory=True
    )

    log.write('** start training here! **\n')
    log.write('\n')
    log.write('epoch    iter      rate     | smooth_loss/dice | valid_loss/dice | best_epoch/best_score |  min \n')
    log.write('------------------------------------------------------------------------------------------------\n')
    start_epoch += 1
    for epoch in range(start_epoch, config['epochs'] + 1):
        end = time.time()

        # set manual seeds per epoch
        np.random.seed(epoch)
        torch.manual_seed(epoch)
        torch.cuda.manual_seed_all(epoch)

        # adjust learning rate for each epoch
        lr_list = scheduler.step(model, epoch, config['epochs'])
        lr = lr_list[0]

        # train for one epoch on train set
        iter, train_loss, train_dice = train(train_loader, model, ema_model, criterion, optimizer, epoch, lr=lr)

        with torch.no_grad():
            if config['ema']:
                valid_loss, valid_dice = validate(valid_loader, ema_model, criterion, epoch)
            else:
                valid_loss, valid_dice = validate(valid_loader, model, criterion, epoch)

        # remember best loss and save checkpoint
        is_best = valid_dice >= best_dice
        if is_best:
            best_epoch = epoch
            best_dice = valid_dice

        if config['ema']:
            save_top_epochs(model_out_dir, ema_model, best_dice_arr, valid_dice,
                            best_epoch, epoch, best_dice, ema=True)
        best_dice_arr = save_top_epochs(model_out_dir, model, best_dice_arr, valid_dice,
                                        best_epoch, epoch, best_dice, ema=False)

        print('\r', end='', flush=True)
        log.write('%5.1f   %5d    %0.6f   |  %0.4f  %0.4f  |  %0.4f  %6.4f |  %6.1f     %6.4f    | %3.1f min \n' % \
                  (epoch, iter + 1, lr, train_loss, train_dice, valid_loss, valid_dice,
                   best_epoch, best_dice, (time.time() - end) / 60))

        model_name = '%03d' % epoch
        if config['ema']:
            save_model(ema_model, model_out_dir, epoch, model_name, best_dice_arr, is_best=is_best,
                       optimizer=optimizer, best_epoch=best_epoch, best_dice=best_dice, ema=True)
        save_model(model, model_out_dir, epoch, model_name, best_dice_arr, is_best=is_best,
                   optimizer=optimizer, best_epoch=best_epoch, best_dice=best_dice, ema=False)

def train(train_loader, model, ema_model, criterion, optimizer, epoch, lr=1e-5):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    dices = AverageMeter()

    # switch to train mode
    model.train()

    if config['pseudo'] is not None and epoch > 1:
        train_loader.dataset.resample_pseudo()

    num_its = len(train_loader)
    end = time.time()
    iter = 0
    print_freq = 1
    for iter, iter_data in enumerate(train_loader, 0):
        # measure data loading time
        data_time.update(time.time() - end)

        # zero out gradients so we can accumulate new ones over batches
        optimizer.zero_grad()

        images, masks, indices = iter_data
        images = Variable(images.cuda())
        masks = Variable(masks.cuda())

        outputs = model(images)
        loss = criterion(outputs, masks)
        # loss = criterion(outputs, masks, epoch=epoch)

        losses.update(loss.item())
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), config['clipnorm'])
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        probs = F.sigmoid(outputs)
        dice = metric(probs, masks)
        dices.update(dice.item())

        if config['ema']:
            if epoch >= config['ema_start']:
                accumulate(ema_model, model, decay=config['ema_decay'])
            else:
                accumulate(ema_model, model, decay=0)

        if (iter + 1) % print_freq == 0 or iter == 0 or (iter + 1) == num_its:
            print('\r%5.1f   %5d    %0.6f   |  %0.4f  %0.4f  | ... ' % \
                  (epoch - 1 + (iter + 1) / num_its, iter + 1, lr, losses.avg, dices.avg), \
                  end='', flush=True)

    return iter, losses.avg, dices.avg

def validate(valid_loader, model, criterion, epoch):
    batch_time = AverageMeter()
    losses = AverageMeter()
    dices = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for it, iter_data in enumerate(valid_loader, 0):
        images, masks, indices = iter_data
        images = Variable(images.cuda())
        masks = Variable(masks.cuda())

        outputs = model(images)
        loss = criterion(outputs, masks)
        # loss = criterion(outputs, masks, epoch=epoch)
        probs = F.sigmoid(outputs)
        dice = metric(probs, masks)

        losses.update(loss.item())
        dices.update(dice.item())

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

    return losses.avg, dices.avg

def save_model(model, model_out_dir, epoch, model_name, best_dice_arr, is_best=False,
               optimizer=None, best_epoch=None, best_dice=None, ema=False):
    if type(model) == DataParallel:
        state_dict = model.module.state_dict()
    else:
        state_dict = model.state_dict()
    for key in state_dict.keys():
        state_dict[key] = state_dict[key].cpu()

    if ema:
        model_fpath = opj(model_out_dir, '%s_ema.pth' % model_name)
    else:
        model_fpath = opj(model_out_dir, '%s.pth' % model_name)
    torch.save({
        'state_dict': state_dict,
        'best_epoch': best_epoch,
        'epoch': epoch,
        'best_dice': best_dice,
        'best_dice_arr': best_dice_arr,
    }, model_fpath)

    optim_fpath = opj(model_out_dir, '%s_optim.pth' % model_name)
    if optimizer is not None:
        torch.save({
            'optimizer': optimizer.state_dict(),
        }, optim_fpath)

    if is_best:
        if ema:
            best_model_fpath = opj(model_out_dir, 'final_ema.pth')
        else:
            best_model_fpath = opj(model_out_dir, 'final.pth')
        shutil.copyfile(model_fpath, best_model_fpath)
        if optimizer is not None:
            best_optim_fpath = opj(model_out_dir, 'final_optim.pth')
            shutil.copyfile(optim_fpath, best_optim_fpath)

def metric(logit, truth, threshold=0.5):
    dice = dice_score(logit, truth, threshold=threshold)
    return dice

def accumulate(model1, model2, decay=0.99):
    par1 = model1.state_dict()
    par2 = model2.state_dict()

    with torch.no_grad():
        for k in par1.keys():
            par1[k].data.copy_(par1[k].data * decay + par2[k].data * (1 - decay))

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.
        self.avg = 0.
        self.sum = 0.
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def save_top_epochs(model_out_dir, model, best_dice_arr, valid_dice, best_epoch, epoch, best_dice, ema=False):
    best_dice_arr = best_dice_arr.copy()

    if ema:
        suffix = '_ema'
    else:
        suffix = ''
    min_dice = np.min(best_dice_arr)
    last_ix = len(best_dice_arr) - 1

    def get_top_path(ix):
        return opj(model_out_dir, 'top%d%s.pth' % (ix + 1, suffix))

    if valid_dice > min_dice:
        min_ix = last_ix
        for ix, score in enumerate(best_dice_arr):
            if score < valid_dice:
                min_ix = ix
                break

        lowest_path = get_top_path(last_ix)
        if ope(lowest_path):
            os.remove(lowest_path)

        for ix in range(last_ix - 1, min_ix - 1, -1):
            score = best_dice_arr[ix]
            best_dice_arr[ix + 1] = score
            if score > 0 and ope(get_top_path(ix)):
                os.rename(get_top_path(ix), get_top_path(ix + 1))

        best_dice_arr[min_ix] = valid_dice

        model_name = 'top%d' % (min_ix + 1)
        save_model(model, model_out_dir, epoch, model_name, best_dice_arr, is_best=False,
                   optimizer=None, best_epoch=best_epoch, best_dice=best_dice, ema=ema)

    return best_dice_arr

In [46]:
print('calling main function ... \n')
main()
print('\nsuccess!')

calling main function ... 

>> Creating directory if it does not exist:
>> '../output/result/models/.'
>> Using pre-trained model.
load pretrained file: ../input/pretrained/resnet34-333f7ec4.pth


FileNotFoundError: [Errno 2] No such file or directory: '../input/pretrained/resnet34-333f7ec4.pth'